# Imports

In [ ]:
from notebooks import *

INFO     [21:34:26.480] [44030] 151 logging_/<module>: {logging_yaml: None}

In [ ]:
# Import bqq() / %%bqq
#   - HACK Add pydatalab
#       - Use local fork to avoid having to deal with upgrades and porting old patches
sys.path.insert(1, os.path.expanduser('~/hack/pydatalab'))
from potoo.bqq import *
%load_ext potoo.sql_magic

In [ ]:
def var_checkpoint(var_name, env, pkl_path='/tmp/var_checkpoint/%s.pkl'):
    pkl_path = pkl_path % var_name
    if var_name in env:
        print('dump: %(var_name)s -> %(pkl_path)s' % locals())
        joblib.dump(env[var_name], ensure_parent_dir(pkl_path))
    else:
        print('load: %(var_name)s <- %(pkl_path)s' % locals())
        env[var_name] = joblib.load(pkl_path)

def sort_like(xs, ys):
    ys = list(ys)
    return sorted(xs, key=lambda x: ys.index(x) if x in ys else -1)

def sort_like_com_name(xs):
    return sort_like(xs, metadata.species.df.com_name.dtype.categories)

def sort_like_species_group(xs):
    return sort_like(xs, metadata.species.df.species_group.dtype.categories)

species_group_for_com_name = (metadata.species.df
    [['com_name', 'species_group']]
    .set_index('com_name')
    .to_dict()['species_group']
)

# bq utils

In [ ]:
# TODO Are these preventing bq cache hits?
defs = '''

    create temp function mul_trunc(x int64, y int64) as (
        div(x, y) * y
    );

    # Mimic ebird_week in ebird_priors.py
    # 'Week' like ebird barchart weeks: 4 weeks per month, where the 4th week includes all days past 28
    #   - 0-indexed (0–47)
    create temp function ebird_week(d date) as (
        (extract(month from d) - 1) * 4 +
        least(div(extract(day from d) - 1, 7), 3)
    );

    # Mimic ebird_bar_width in ebird_priors.py
    # https://help.ebird.org/customer/portal/articles/1010553-understanding-the-ebird-bar-charts
    # https://help.ebird.org/customer/en/portal/articles/1210247-what-is-frequency-
    create temp function ebird_bar_width(p float64) as (
        case
            when p >= .600 then 9
            when p >= .400 then 8
            when p >= .300 then 7
            when p >= .200 then 6
            when p >= .100 then 5
            when p >= .050 then 4
            when p >= .010 then 3
            when p >= .003 then 2
            when p >= .000 then 1
        end
    );

    create temp function _geohash_char_at_bits(c string, char_bits int64) as (
        # https://en.wikipedia.org/wiki/Geohash
        substr(
            case char_bits
                when 0 then '0123456789bcdefghjkmnpqrstuvwxyz'
                when 1 then '0000000000000000hhhhhhhhhhhhhhhh'
                when 2 then '0000000088888888hhhhhhhhssssssss'
                when 3 then '000044448888ddddhhhhnnnnsssswwww'
                when 4 then '0022446688bbddffhhkknnqqssuuwwyy'
            end,
            strpos('0123456789bcdefghjkmnpqrstuvwxyz', c),
            1
        )
    );

    create temp function geohash_at_bits(geo geography, bits int64) as (
        case
            # Guard against bits > 100 since st_geohash returns max 20 chars (* 5 = 100 bits)
            when not (0 < bits and bits <= 100) then error(format('Expected 0 < bits[%d] <= 100', bits))
            else concat(
                # All but last char in geohash ((bits - 1) // 5)
                #   - Manually substr(st_geohash(geo, 20), ...) to avoid st_geohash(geo, 0), which returns all 20 bits
                substr(st_geohash(geo, 20), 1, div(bits - 1, 5)),
                # Last char in geohash, truncated to remaining bits (% 5)
                _geohash_char_at_bits(
                    substr(st_geohash(geo, div(bits - 1, 5) + 1), -1, 1),
                    mod(bits, 5)
                )
            )
        end
    );

'''

# Inspect defs
#   - TODO Add some asserts
(bqq(defs=defs, sql='''
    select
        common_name,
        ebird_week(observation_date) as observation_week,
        locality_id,
        locality,
        st_geohash(st_geogpoint(longitude, latitude))           as n_all,
        st_geohash(st_geogpoint(longitude, latitude), 20)       as n_20,
        geohash_at_bits(st_geogpoint(longitude, latitude), 100) as b_100,
        geohash_at_bits(st_geogpoint(longitude, latitude), 99)  as b_99,
        geohash_at_bits(st_geogpoint(longitude, latitude), 98)  as b_98,
        geohash_at_bits(st_geogpoint(longitude, latitude), 97)  as b_97,
        geohash_at_bits(st_geogpoint(longitude, latitude), 96)  as b_96,
        st_geohash(st_geogpoint(longitude, latitude), 19)       as n_19,
        geohash_at_bits(st_geogpoint(longitude, latitude), 95)  as b_95,
        geohash_at_bits(st_geogpoint(longitude, latitude), 94)  as b_94,
        geohash_at_bits(st_geogpoint(longitude, latitude), 93)  as b_93,
        geohash_at_bits(st_geogpoint(longitude, latitude), 92)  as b_92,
        geohash_at_bits(st_geogpoint(longitude, latitude), 91)  as b_91,
        st_geohash(st_geogpoint(longitude, latitude), 2)        as n_2,
        geohash_at_bits(st_geogpoint(longitude, latitude), 10)  as b_10,
        geohash_at_bits(st_geogpoint(longitude, latitude), 9)   as b_9,
        geohash_at_bits(st_geogpoint(longitude, latitude), 8)   as b_8,
        geohash_at_bits(st_geogpoint(longitude, latitude), 7)   as b_7,
        geohash_at_bits(st_geogpoint(longitude, latitude), 6)   as b_6,
        st_geohash(st_geogpoint(longitude, latitude), 1)        as n_1,
        geohash_at_bits(st_geogpoint(longitude, latitude), 5)   as b_5,
        geohash_at_bits(st_geogpoint(longitude, latitude), 4)   as b_4,
        geohash_at_bits(st_geogpoint(longitude, latitude), 3)   as b_3,
        geohash_at_bits(st_geogpoint(longitude, latitude), 2)   as b_2,
        geohash_at_bits(st_geogpoint(longitude, latitude), 1)   as b_1,
        # count(*) as n,
        ''
    from ebird.obs_sample_001
    where true
        and state_code = 'US-CA'
        and category in ('species', 'issf', 'form', 'intergrade')
    # group by 1,2,3,4,5,6,7,8,9
    order by rand()
    limit 20
''')
    .T
)

Running query...

[5s] cost[$0.0002, 48.6 MB] rows[20] size[0 B] url[https://console.cloud.google.com/bigquery?project=bubo-1&j=bq:US:job_OuooxaXaplVIZ3wesHdp39r01FkB&page=queryresults]

Fetching results...

[0s]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
common_name,White-crowned Sparrow,Cliff Swallow,Bufflehead,Black Phoebe,Brown Pelican,Greater White-fronted Goose,Osprey,Dark-eyed Junco,American Kestrel,Varied Thrush,White-throated Sparrow,Red-tailed Hawk,Cassin's Vireo,Mourning Dove,Orange-crowned Warbler,Black-necked Stilt,Hairy Woodpecker,Wilson's Warbler,Band-tailed Pigeon,Ruby-crowned Kinglet
observation_week,5,21,8,47,36,3,18,10,43,43,3,15,18,17,17,20,26,19,21,0
locality_id,L247968,L3615819,L2698912,L6617340,L351583,L168785,L481707,L4397650,L370676,L661996,L468377,L1072372,L249749,L2281623,L419660,L2288311,L350696,L824120,L951033,L271485
locality,San Bruno Mountain Park,Edwards AFB--Piute Ponds: Point 2,Ed Levin County Park--Sandy Wool area,"east end of Village Lane, Cambria, California, US (35.564, -121.079)",Ventura Harbor,Sacramento NWR,Tomales Bay--Cypress Grove Pt.,"Rambler Trail, Clover Valley Preserve, Grass Valley",Moss Landing--town or marina,Woodlawn Cemetery (LA Co.),Lodi Lake Nature Area,UC Santa Barbara--Lagoon,San Luis NWR,"6948 Corte Verde, Pleasanton, CA",Desert Center--Lake Tamarisk,Crittenden Marsh / Moffett Field,Pacific Grove--yard 282 Grove Acre,Kyburz Flat area,SCZ FBMP BONN 13,Arcata Marsh--Oxidation Ponds
n_all,9q8ymyvhgm45jdxfxtrd,9qhjbq3d3r246jttgxuw,9q9m4ftuts848bche9j6,9q3fkf16yn870edeycqz,9q54y4bjq1200y3jhent,9r105vt1k1m1yh5wdt71,9qb95k1bk08427dkphpf,9qcyt9rgn9byr8h9fque,9q93kevf39s4rcyprb13,9q5c80b244b4cp7scxfn,9qc9pcjxegxgr1n0t0wf,9q4g8z9h1brypb7kg2wu,9qd5ehh1eyx7tu13wwsg,9q9q3cnz9uh42t3htdrr,9mvzynhp6qxbxmg4cyqx,9q9hym7wfctrj70nx2dx,9q92377vrks8c55u89p1,9r42xzebk12n0e7kvw6k,9q94sjqfndrbqvx1x1fg,9prb2k9p3489sxuygtss
n_20,9q8ymyvhgm45jdxfxtrd,9qhjbq3d3r246jttgxuw,9q9m4ftuts848bche9j6,9q3fkf16yn870edeycqz,9q54y4bjq1200y3jhent,9r105vt1k1m1yh5wdt71,9qb95k1bk08427dkphpf,9qcyt9rgn9byr8h9fque,9q93kevf39s4rcyprb13,9q5c80b244b4cp7scxfn,9qc9pcjxegxgr1n0t0wf,9q4g8z9h1brypb7kg2wu,9qd5ehh1eyx7tu13wwsg,9q9q3cnz9uh42t3htdrr,9mvzynhp6qxbxmg4cyqx,9q9hym7wfctrj70nx2dx,9q92377vrks8c55u89p1,9r42xzebk12n0e7kvw6k,9q94sjqfndrbqvx1x1fg,9prb2k9p3489sxuygtss
b_100,9q8ymyvhgm45jdxfxtrd,9qhjbq3d3r246jttgxuw,9q9m4ftuts848bche9j6,9q3fkf16yn870edeycqz,9q54y4bjq1200y3jhent,9r105vt1k1m1yh5wdt71,9qb95k1bk08427dkphpf,9qcyt9rgn9byr8h9fque,9q93kevf39s4rcyprb13,9q5c80b244b4cp7scxfn,9qc9pcjxegxgr1n0t0wf,9q4g8z9h1brypb7kg2wu,9qd5ehh1eyx7tu13wwsg,9q9q3cnz9uh42t3htdrr,9mvzynhp6qxbxmg4cyqx,9q9hym7wfctrj70nx2dx,9q92377vrks8c55u89p1,9r42xzebk12n0e7kvw6k,9q94sjqfndrbqvx1x1fg,9prb2k9p3489sxuygtss
b_99,9q8ymyvhgm45jdxfxtrd,9qhjbq3d3r246jttgxuw,9q9m4ftuts848bche9j6,9q3fkf16yn870edeycqy,9q54y4bjq1200y3jhens,9r105vt1k1m1yh5wdt70,9qb95k1bk08427dkphpf,9qcyt9rgn9byr8h9fqud,9q93kevf39s4rcyprb12,9q5c80b244b4cp7scxfn,9qc9pcjxegxgr1n0t0wf,9q4g8z9h1brypb7kg2wu,9qd5ehh1eyx7tu13wwsf,9q9q3cnz9uh42t3htdrq,9mvzynhp6qxbxmg4cyqw,9q9hym7wfctrj70nx2dw,9q92377vrks8c55u89p0,9r42xzebk12n0e7kvw6k,9q94sjqfndrbqvx1x1ff,9prb2k9p3489sxuygtss
b_98,9q8ymyvhgm45jdxfxtrd,9qhjbq3d3r246jttgxuw,9q9m4ftuts848bche9j4,9q3fkf16yn870edeycqw,9q54y4bjq1200y3jhens,9r105vt1k1m1yh5wdt70,9qb95k1bk08427dkphpd,9qcyt9rgn9byr8h9fqud,9q93kevf39s4rcyprb10,9q5c80b244b4cp7scxfn,9qc9pcjxegxgr1n0t0wd,9q4g8z9h1brypb7kg2ws,9qd5ehh1eyx7tu13wwsd,9q9q3cnz9uh42t3htdrn,9mvzynhp6qxbxmg4cyqw,9q9hym7wfctrj70nx2dw,9q92377vrks8c55u89p0,9r42xzebk12n0e7kvw6h,9q94sjqfndrbqvx1x1fd,9prb2k9p3489sxuygtss
b_97,9q8ymyvhgm45jdxfxtr8,9qhjbq3d3r246jttgxus,9q9m4ftuts848bche9j0,9q3fkf16yn870edeycqs,9q54y4bjq1200y3jhens,9r105vt1k1m1yh5wdt70,9qb95k1bk08427dkphp8,9qcyt9rgn9byr8h9fqu8,9q93kevf39s4rcyprb10,9q5c80b244b4cp7scxfh,9qc9pcjxegxgr1n0t0w8,9q4g8z9h1brypb7kg2ws,9qd5ehh1eyx7tu13wws8,9q9q3cnz9uh42t3htdrh,9mvzynhp6qxbxmg4cyqs,9q9hym7wfctrj70nx2ds,9q92377vrks8c55u89p0,9r42xzebk12n0e7kvw6h,9q94sjqfndrbqvx1x1f8,9prb2k9p3489sxuygtss


In [ ]:
# For join with metadata.ebird.shorthand, ensure no/few species unmapped species
#   - cf. model.metadata.com_names_to_species
(bqq('''
    with ebird_species as (
        select distinct
            common_name,
            category
        from ebird.obs_all_expensive
        where true
            and category in ('species', 'issf', 'form', 'intergrade')
            and country_code = 'US'
    )
    select
        category,
        common_name,
        com_name,
        shorthand
    from ebird_species e
        left join metadata.species s on (e.common_name = s.com_name)
    where true
        and (common_name is null or com_name is null)
''')
    .pipe(df_inspect, lambda df: (df.shape,))
    .fillna('')
)

Running query...

[2s] cost[cached] rows[2] size[0 B] url[https://console.cloud.google.com/bigquery?project=bubo-1&j=bq:US:job_S3-PifTDBKiV7M_dd1-wfvJFZggE&page=queryresults]

Fetching results...

[0s]

(2, 4)

,category,common_name,com_name,shorthand
0,form,Leach's/Townsend's Storm-Petrel (dark-rumped),,
1,form,Leach's/Townsend's Storm-Petrel (white-rumped),,


# Basic stats

In [ ]:
# WARNING $0.09
# How many obs, checklists, hotspots, species?
(bqq('''
    select
        count(*)                                  as n,
        count(distinct sampling_event_identifier) as n_checklists,
        count(distinct locality_id)               as n_hotspots,
        count(distinct common_name)               as n_species,
        ''
    from ebird.obs_all_expensive
''')
    .T
)

Running query...

[1s] cost[cached] rows[1] url[https://console.cloud.google.com/bigquery?project=bubo-1&j=bq:US:job_P_-fB-hEzYXy1c6cdXk7WbhTnKMy&page=queryresults]

Fetching results...

[1s]

,0
n,508111445
n_checklists,36812628
n_hotspots,5710382
n_species,11726
f0_,


In [ ]:
# WARNING $0.01
# How many obs by country?
#   - 362m/508m in US
#   -  50m/508m is next biggest country
bqq('''
    select
        country_code,
        count(*) n_obs,
        sum(count(*)) over () total_obs
    from ebird.obs_all_expensive
    group by 1
    order by 2 desc
    limit 10
''')

Running query...

[3s] cost[$0.0092, 2.0 GB] rows[10] url[https://console.cloud.google.com/bigquery?project=bubo-1&j=bq:US:job_puyHAyw2hV3s481S0FqgF08oaXxd&page=queryresults]

Fetching results...

[1s]

,country_code,n_obs,total_obs
0,US,362480201,508111445
1,CA,50217998,508111445
2,AU,14418719,508111445
3,IN,10349707,508111445
4,MX,6293521,508111445
5,CR,4931603,508111445
6,GB,4471881,508111445
7,ES,3897594,508111445
8,PT,3345458,508111445
9,BR,3210548,508111445


In [ ]:
# WARNING $0.04
# How many unique geohashes, at various precisions?
#   - See ebird_priors.py for bits:mi mapping (EbirdPriors._loc_binwidths)
(bqq(defs=defs, sql='''
    select
        count(*)                                                               as n,
        count(distinct geohash_at_bits(st_geogpoint(longitude, latitude), 31)) as n_geohash_0_4mi,
        count(distinct geohash_at_bits(st_geogpoint(longitude, latitude), 29)) as n_geohash_0_8mi,
        count(distinct geohash_at_bits(st_geogpoint(longitude, latitude), 27)) as n_geohash_1_5mi,
        count(distinct geohash_at_bits(st_geogpoint(longitude, latitude), 25)) as n_geohash_3mi,
        count(distinct geohash_at_bits(st_geogpoint(longitude, latitude), 23)) as n_geohash_6mi,
        count(distinct geohash_at_bits(st_geogpoint(longitude, latitude), 21)) as n_geohash_12mi,
        count(distinct geohash_at_bits(st_geogpoint(longitude, latitude), 19)) as n_geohash_24mi,
        count(distinct geohash_at_bits(st_geogpoint(longitude, latitude), 17)) as n_geohash_48mi,
        count(distinct geohash_at_bits(st_geogpoint(longitude, latitude), 15)) as n_geohash_97mi,
        count(distinct geohash_at_bits(st_geogpoint(longitude, latitude), 13)) as n_geohash_194mi,
        ''
    from ebird.obs_all_expensive
''')
    .T
)

Running query...

[34s] cost[$0.0370, 8.1 GB] rows[1] url[https://console.cloud.google.com/bigquery?project=bubo-1&j=bq:US:job_L28ABB9d5C10VhwDEa_I3I55809e&page=queryresults]

Fetching results...

[1s]

,0
n,508111445
n_geohash_0_4mi,2946715
n_geohash_0_8mi,2041719
n_geohash_1_5mi,1239658
n_geohash_3mi,666661
n_geohash_6mi,324744
n_geohash_12mi,146243
n_geohash_24mi,62410
n_geohash_48mi,25512
n_geohash_97mi,10065


# Priors: Start with something simple

In [ ]:
# Inspect available tables
!bq ls metadata
!bq ls xc
!bq ls ebird

  tableId   Type    Labels   Time Partitioning  
 --------- ------- -------- ------------------- 
  species   TABLE                               


           tableId             Type    Labels   Time Partitioning  
 ---------------------------- ------- -------- ------------------- 
  _rec_count_per_species_csv   TABLE                               
  n_recs_per_species           VIEW                                


            tableId              Type    Labels   Time Partitioning  
 ------------------------------ ------- -------- ------------------- 
  _ebd_relMay_2018_txt           TABLE                               
  _ebd_relMay_2018_txt_001       TABLE                               
  _ebd_relMay_2018_txt_1000      TABLE                               
  _ebd_relMay_2018_txt_1000000   TABLE                               
  _ws1_1_taxa_species_csv        TABLE                               
  checklist_species_us_az        TABLE                               
  checklist_species_us_az_trip   TABLE                               
  obs_all_expensive              VIEW                                
  obs_sample_001                 VIEW                                
  ws1_1_taxa_species             VIEW                                


In [ ]:
# Inspect rows
bqq('''
    select *
    from ebird.obs_sample_001
    where true
        and state_code = 'US-CA'
        and category in ('species', 'issf', 'form', 'intergrade')
    limit 20
''')

Running query...

[3s] cost[$0.0009, 203.6 MB] rows[20] url[https://console.cloud.google.com/bigquery?project=bubo-1&j=bq:US:job_2rqkj81cTvvGxtHq3jTmBIHeBWWl&page=queryresults]

Fetching results...

[1s]

,global_unique_identifier,last_edited_date,taxonomic_order,category,common_name,scientific_name,subspecies_common_name,subspecies_scientific_name,observation_count,breeding_bird_atlas_code,breeding_bird_atlas_category,age_sex,country,country_code,state,state_code,county,county_code,iba_code,bcr_code,usfws_code,atlas_block,locality,locality_id,_locality_type,latitude,longitude,observation_date,time_observations_started,observer_id,sampling_event_identifier,protocol_type,protocol_code,project_code,duration_minutes,effort_distance_km,effort_area_ha,number_observers,all_species_reported,group_identifier,has_media,approved,reviewed,reason,trip_comments,species_comments
0,URN:CornellLabOfOrnithology:EBIRD:OBS169396759,2016-08-09 15:15:45,30000.0,species,White-crowned Sparrow,Zonotrichia leucophrys,None,None,20,None,None,None,United States,US,California,US-CA,Alameda,US-CA-001,None,NaN,None,None,San Lorenzo Trail-Grant Ave,L1803364,P,37.7,-122.0,2012-11-11,10:00:00,obsr293948,S12020443,Traveling,P22,EBIRD,25.0,0.201,NaN,3.0,1,G576066,0,1,0,None,None,None
1,URN:CornellLabOfOrnithology:EBIRD:OBS152695281,2016-08-09 15:25:47,8910.0,species,Anna's Hummingbird,Calypte anna,None,None,1,None,None,None,United States,US,California,US-CA,Contra Costa,US-CA-013,None,32.0,None,None,EBMUD--Valle Vista Staging Area (permit required),L596210,H,37.8,-122.0,2012-05-13,09:20:00,obsr58810,S10734725,Traveling,P22,EBIRD,65.0,0.483,NaN,1.0,1,None,0,1,0,None,None,None
2,URN:CornellLabOfOrnithology:EBIRD:OBS158990811,2012-07-18 16:53:58,14900.0,species,Ash-throated Flycatcher,Myiarchus cinerascens,None,None,1,None,None,None,United States,US,California,US-CA,El Dorado,US-CA-017,None,15.0,None,None,Mt. Tallac Trail,L1627922,P,38.9,-120.0,2012-07-17,09:20:00,obsr150458,S11174533,Incidental,P20,EBIRD,NaN,NaN,NaN,2.0,1,None,0,1,0,None,Hike to Mt. Tallac via Mt. Tallac Trail,None
3,URN:CornellLabOfOrnithology:EBIRD:OBS147837396,2017-08-16 08:21:29,30900.0,species,House Finch,Haemorhous mexicanus,None,None,X,None,None,None,United States,US,California,US-CA,Humboldt,US-CA-023,US-CA_43,5.0,None,None,Arcata Marsh--Brackish Pond,L772156,H,40.9,-124.0,2012-04-09,19:00:00,obsr252573,S10410161,Stationary,P21,EBIRD,20.0,NaN,NaN,1.0,1,None,0,1,0,None,None,None
4,URN:CornellLabOfOrnithology:EBIRD:OBS174141474,2016-08-09 15:37:46,11200.0,species,Hairy Woodpecker,Picoides villosus,None,None,1,None,None,None,United States,US,California,US-CA,Humboldt,US-CA-023,None,5.0,None,None,Trinity River Farm,L1854380,H,41.0,-124.0,2012-12-27,12:20:00,obsr27015,S12403128,Area,P23,EBIRD,130.0,NaN,6.0,2.0,1,None,0,1,0,None,Rob Hewitt and I spent a lot of quality time here searching it a second time after David Fix and Jude Power visited in the morning. A great spot for birding with LOTS of sp...,None
5,URN:CornellLabOfOrnithology:EBIRD:OBS168359167,2012-11-03 21:11:12,11100.0,species,Downy Woodpecker,Picoides pubescens,None,None,1,None,None,None,United States,US,California,US-CA,Inyo,US-CA-027,None,9.0,None,None,Bishop City Park,L812982,H,37.4,-118.0,2012-11-02,10:00:00,obsr204942,S11948072,Traveling,P22,EBIRD,191.0,0.805,NaN,7.0,1,G484136,0,1,0,None,"w/ Overholtzes, Howards & JLD",None
6,URN:CornellLabOfOrnithology:EBIRD:OBS163369388,2012-09-10 21:22:49,10900.0,species,Red-breasted Sapsucker,Sphyrapicus ruber,None,None,1,None,None,None,United States,US,California,US-CA,Inyo,US-CA-027,None,15.0,None,None,North Lake (INY Co.),L955403,H,37.2,-119.0,2012-09-08,06:55:00,obsr266240,S11557977,Traveling,P22,EBIRD,462.0,16.100,NaN,1.0,1,G462502,0,1,0,None,"Hike from north lake to Emerson lake Submitted from BirdLog NA for iOS, version 1.4.8",None
7,URN:CornellLabOfOrnithology:EBIRD:OBS155916886,2016-03-16 16:34:18,17800.0,species,Warbling Vireo,Vireo gilvus,None,None,1,None,None,None,United States,US,California,US-CA,Inyo,US-CA-027,None,33.0,None,None,"Mesquite Springs CG, Death VAlley CA",L1577199,P,37.0,-117.0,2012-05-08,06:10:00,obsr40579,S10949032,Traveling,P22,EBIRD,140.0,0.402,NaN,2.0,1,None,

In [ ]:
# XXX
bqq('''
    select
        count(*),
        count(distinct sampling_event_identifier),
        ''
    from ebird.obs_sample_001
    where true
        and st_distance(st_geogpoint(longitude, latitude), st_geogpoint(-122.4194, 37.7749)) <= 20e3 # SF ± 20km
        and category in ('species', 'issf', 'form', 'intergrade')
''')

Running query...

[1s] cost[cached] rows[1] url[https://console.cloud.google.com/bigquery?project=bubo-1&j=bq:US:job_e5muhE7ZY9uoogVnKq0cOOJ44I9q&page=queryresults]

Fetching results...

[1s]

,f0_,f1_,f2_
0,2381,2344,


In [ ]:
(bqq('''
    with
        obs as (
            select distinct
                common_name,
                sampling_event_identifier,
                count(distinct common_name) over () as n_species,
                count(distinct sampling_event_identifier) over () as n_checklists
            from ebird.obs_sample_001
            where true
                and category in ('species', 'issf', 'form', 'intergrade')
                and st_distance(st_geogpoint(longitude, latitude), st_geogpoint(-122.4194, 37.7749)) <= 20e3 # SF ± 20km
        )
    select
        n_species,
        n_checklists,
        common_name,
        count(*) as n_present,
        count(*) / n_checklists as p_present,
        ''
    from obs
    group by n_species, n_checklists, common_name
    order by common_name
''')
    .pipe(df_display, lambda df: (df.shape,))
    .assign(ebird_bar_width=lambda df: df.p_present.map(ebird_bar_width))
    .assign(ebird_bar_str=lambda df: df.p_present.map(ebird_bar_str))
    .assign(p_present=lambda df: df.p_present.map(lambda x: '%.3f' % x))
    .sort_values('p_present', ascending=False)
    [:20]
)

Running query...

[4s] cost[$0.0001, 27.2 MB] rows[209] url[https://console.cloud.google.com/bigquery?project=bubo-1&j=bq:US:job_2YIGbbQ8fymg593Ffx6lF37aLEZo&page=queryresults]

Fetching results...

[1s]

(209, 6)

,n_species,n_checklists,common_name,n_present,p_present,f0_,ebird_bar_width,ebird_bar_str
197,209,2344,White-crowned Sparrow,79,0.034,,3,■■■——————
12,209,2344,Anna's Hummingbird,74,0.032,,3,■■■——————
99,209,2344,House Finch,61,0.026,,3,■■■——————
114,209,2344,Mallard,56,0.024,,3,■■■——————
4,209,2344,American Crow,56,0.024,,3,■■■——————
19,209,2344,Black Phoebe,55,0.023,,3,■■■——————
190,209,2344,Western Gull,52,0.022,,3,■■■——————
60,209,2344,Common Raven,46,0.020,,3,■■■——————
42,209,2344,California Towhee,47,0.020,,3,■■■——————
170,209,2344,Song Sparrow,46,0.020,,3,■■■——————


In [ ]:
# WARNING $0.12
(bqq('''
    with
        obs as (
            select distinct
                common_name,
                sampling_event_identifier,
                count(distinct common_name) over () as n_species,
                count(distinct sampling_event_identifier) over () as n_checklists
            from ebird.obs_all_expensive
            where true
                and category in ('species', 'issf', 'form', 'intergrade')
                and st_distance(st_geogpoint(longitude, latitude), st_geogpoint(-122.4194, 37.7749)) <= 20e3 # SF ± 20km
        )
    select
        n_species,
        n_checklists,
        common_name,
        count(*) as n_present,
        count(*) / n_checklists as p_present,
        ''
    from obs
    group by n_species, n_checklists, common_name
    order by common_name
''')
    .pipe(df_display, lambda df: (df.shape,))
    .assign(ebird_bar_width=lambda df: df.p_present.map(ebird_bar_width))
    .assign(ebird_bar_str=lambda df: df.p_present.map(ebird_bar_str))
    .assign(p_present=lambda df: df.p_present.map(lambda x: '%.3f' % x))
    .sort_values('p_present', ascending=False)
    [:20]
)

Running query...

[11s] cost[$0.1233, 27.1 GB] rows[489] url[https://console.cloud.google.com/bigquery?project=bubo-1&j=bq:US:job_Lm7y_WNOI36Hov_Fi-tUxVwVQp8t&page=queryresults]

Fetching results...

[0s]

(489, 6)

,n_species,n_checklists,common_name,n_present,p_present,f0_,ebird_bar_width,ebird_bar_str
20,489,150611,Anna's Hummingbird,68096,0.452,,8,■■■■■■■■—
451,489,150611,White-crowned Sparrow,59776,0.397,,7,■■■■■■■——
440,489,150611,Western Gull,59595,0.396,,7,■■■■■■■——
219,489,150611,House Finch,55743,0.370,,7,■■■■■■■——
6,489,150611,American Crow,53830,0.357,,7,■■■■■■■——
40,489,150611,Black Phoebe,49936,0.332,,7,■■■■■■■——
403,489,150611,Song Sparrow,49160,0.326,,7,■■■■■■■——
102,489,150611,California Towhee,47628,0.316,,7,■■■■■■■——
139,489,150611,Common Raven,43418,0.288,,6,■■■■■■———
14,489,150611,American Robin,42922,0.285,,6,■■■■■■———


# How about small, useful intermediate data for the app?

In [ ]:
bqq('''
    select *
    from ebird.obs_sample_001
    where true
        and state_code = 'US-CA'
        and category in ('species', 'issf', 'form', 'intergrade')
    order by rand()
    limit 10
''')

Running query...

[4s] cost[$0.0009, 203.6 MB] rows[10] url[https://console.cloud.google.com/bigquery?project=bubo-1&j=bq:US:job_OUnt5yOja-M3BGSeVTUbtdOS-7zg&page=queryresults]

Fetching results...

[0s]

,global_unique_identifier,last_edited_date,taxonomic_order,category,common_name,scientific_name,subspecies_common_name,subspecies_scientific_name,observation_count,breeding_bird_atlas_code,breeding_bird_atlas_category,age_sex,country,country_code,state,state_code,county,county_code,iba_code,bcr_code,usfws_code,atlas_block,locality,locality_id,_locality_type,latitude,longitude,observation_date,time_observations_started,observer_id,sampling_event_identifier,protocol_type,protocol_code,project_code,duration_minutes,effort_distance_km,effort_area_ha,number_observers,all_species_reported,group_identifier,has_media,approved,reviewed,reason,trip_comments,species_comments
0,URN:CornellLabOfOrnithology:EBIRD:OBS274081873,2016-08-10 14:57:28,14600.0,species,Pacific-slope Flycatcher,Empidonax difficilis,None,None,1,None,None,None,United States,US,California,US-CA,Ventura,US-CA-111,None,32.0,None,None,Arroyo Verde Park,L1579832,P,34.3,-119.0,2014-09-30,08:30:00,obsr290167,S19990274,Traveling,P22,EBIRD,75.0,1.610,NaN,2,1,None,0,1,0,None,None,None
1,URN:CornellLabOfOrnithology:EBIRD:OBS214008504,2016-04-05 16:21:53,27800.0,species,Phainopepla,Phainopepla nitens,None,None,2,None,None,None,United States,US,California,US-CA,Amador,US-CA-005,None,32.0,None,None,Ione WTP,L486218,H,38.4,-121.0,2013-10-13,11:26:00,obsr147985,S15394534,Traveling,P22,EBIRD,22.0,0.161,NaN,1,1,None,0,1,0,None,Submitted from BirdLog NA for Android v1.9.2,None
2,URN:CornellLabOfOrnithology:EBIRD:OBS27059837,2017-08-16 01:39:05,373.0,species,Cinnamon Teal,Spatula cyanoptera,None,None,12,None,None,None,United States,US,California,US-CA,Humboldt,US-CA-023,US-CA_43,5.0,None,None,HUM_Arcata Marsh,L1863148,P,40.9,-124.0,1999-04-11,None,obsr86404,S2042189,Incidental,P20,EBIRD,NaN,NaN,NaN,1,1,None,0,1,0,None,"Birded most of the day, but don''t have hours.",None
3,URN:CornellLabOfOrnithology:EBIRD:OBS298884756,2017-08-16 16:41:15,2820.0,species,Cooper's Hawk,Accipiter cooperii,None,None,1,None,None,None,United States,US,California,US-CA,San Diego,US-CA-073,None,32.0,None,None,Tijuana River Valley Regional Park--Bird & Butterfly Garden,L418752,H,32.6,-117.0,2015-02-21,06:30:00,obsr30312,S22014927,Traveling,P22,EBIRD,103.0,1.130,NaN,2,1,None,0,1,0,None,"0630-0650 (cloudy, calm, ca. 55); 1037-1200 (mostly cloudy; w/2-10; ca. 70) with Guy McCaskie); garden area early; then on return trip spent most time looking for E. Phoeb...",None
4,URN:CornellLabOfOrnithology:EBIRD:OBS286434592,2017-08-16 16:04:54,2390.0,species,White-tailed Kite,Elanus leucurus,None,None,1,None,None,None,United States,US,California,US-CA,Ventura,US-CA-111,US-CA_195,32.0,None,None,Lake Casitas,L297814,H,34.4,-119.0,2014-12-21,09:00:00,obsr447841,S20979049,Traveling,P22,EBIRD,300.0,3.220,NaN,1,1,None,0,1,0,None,"Lakshore from boat ramp to southeastern corner, and oaks around main entrance",None
5,URN:CornellLabOfOrnithology:EBIRD:OBS570482603,2018-01-23 12:52:49,30000.0,species,Dark-eyed Junco,Junco hyemalis,None,None,5,None,None,None,United States,US,California,US-CA,Napa,US-CA-055,None,32.0,None,None,"1505–1723 Partrick Rd, Napa US-CA (38.3152,-122.3706)",L6766592,P,38.3,-122.0,2018-01-23,09:44:00,obsr555822,S42180738,Stationary,P21,EBIRD,5.0,NaN,NaN,2,1,None,0,1,0,None,None,None
6,URN:CornellLabOfOrnithology:EBIRD:OBS119793508,2014-01-02 14:27:18,2070.0,species,Brown Pelican,Pelecanus occidentalis,None,None,24,None,None,None,United States,US,California,US-CA,Monterey,US-CA-053,US-CA_142,NaN,None,None,Elkhorn Slough Reserve (NERR),L353255,H,36.8,-122.0,2011-05-25,11:30:00,obsr88430,S8325389,Traveling,P22,EBIRD,120.0,1.610,NaN,2,1,None,0,1,0,None,Elkhorn Slough Safari boat trip.,None
7,URN:CornellLabOfOrnithology:EBIRD:OBS219023434,2013-11-28 01:47:06,26500.0,species,Northern Mockingbird,Mimus polyglottos,None,None,3,None,None,None,United States,US,California,US-CA,Santa Cruz,US-CA-087,None,32.0,None,None,Santa Cruz- San Juan Ave. near Prospect Heights,L2062852,P,37.0,-122.0,2013-11-27,07:05:00,obsr48945,S1577495

In [ ]:
# WARNING ~$0.20
# Make priors table
#   - See ebird_priors.py for bits:mi mapping (EbirdPriors._loc_binwidths)
# Stats: all countries
#   - geo[12mi] date[1w] country[*]:  115s,  74m rows, 3.36gb
#   - geo[48mi] date[2w] country[*]:   92s,  29m rows, 1.29gb
# Stats: US
#   - geo[12mi] date[1w] country[US]:  85s,  39m rows -- export fails b/c >1gb; could fix with wildcard but sqlite would be too big anyway
#   - geo[12mi] date[2w] country[US]:  64s,  27m rows, bq[937mb,  37s] fetch[181s] df[3.6gb] sqlite[637mb, 147s]
#   - geo[24mi] date[2w] country[US]:  42s,  18m rows, bq[606mb,  35s] fetch[117s] df[2.4gb] sqlite[412mb,  89s]
#   - geo[48mi] date[2w] country[US]:  46s,  13m rows, bq[433mb,  34s] fetch[ 82s] df[1.7gb] sqlite[300mb,  69s]
#   - geo[12mi] date[4w] country[US]:  65s,  18m rows, bq[629mb,  38s] fetch[118s] df[2.4gb] sqlite[429mb,  98s]
#   - geo[24mi] date[4w] country[US]:  44s,  11m rows, bq[392mb,  29s] fetch[ 77s] df[1.6gb] sqlite[267mb,  64s]
#   - geo[48mi] date[4w] country[US]: 106s, 7.9m rows, bq[272mb,  36s] fetch[ 57s] df[1.1gb] sqlite[189mb,  51s]
#   - TODO TODO Go with ?
priors_df = (bqq(defs=defs, sql='''
    with
        a as (
            select

                # geohash_at_bits(st_geogpoint(longitude, latitude), 21) as geohash_bin,          -- 12mi
                # geohash_at_bits(st_geogpoint(longitude, latitude), 19) as geohash_bin,          -- 24mi
                geohash_at_bits(st_geogpoint(longitude, latitude), 17) as geohash_bin,          -- 48mi

                # mul_trunc(ebird_week(observation_date), 1)             as observation_date_bin, -- 1w
                # mul_trunc(ebird_week(observation_date), 2)             as observation_date_bin, -- 2w
                mul_trunc(ebird_week(observation_date), 4)             as observation_date_bin, -- 4w

                ifnull(s.shorthand, '_UNK')                            as species,
                sampling_event_identifier                              as checklist_id,
                min(country_code)                                      as debug_country,
                min(state_code)                                        as debug_state,
                min(county_code)                                       as debug_county,
                min(extract(month from observation_date))              as debug_month,
                count(*)                                               as n

            # from ebird.obs_sample_001 e # XXX Dev
            from ebird.obs_all_expensive e # WARNING

                left join metadata.species s on (s.com_name = e.common_name)
            where true
                and category in ('species', 'issf', 'form', 'intergrade')
                and country_code = 'US'
                # and state_code = 'US-CA' # XXX
                # and county_code = 'US-CA-075' # XXX SF
            group by 1,2,3,4
        ),
        b as (
            select distinct
                geohash_bin,
                observation_date_bin,
                debug_country,
                debug_state,
                debug_county,
                debug_month,
                species,
                count(distinct checklist_id) over (partition by observation_date_bin, geohash_bin)          as bin_n_checklists,
                count(distinct checklist_id) over (partition by observation_date_bin, geohash_bin, species) as bin_n_checklists_with_sp
            from a
        ),
        c as (
            select
                *,
                bin_n_checklists_with_sp / bin_n_checklists as bin_p_sp,
                ebird_bar_width(bin_n_checklists_with_sp / bin_n_checklists) as bin_p_sp_bar_width
            from b
        ),
        d as (
            select
                # *
                geohash_bin,
                observation_date_bin,
                bin_n_checklists,
                bin_n_checklists_with_sp,
                species
            from c
            where true
                # and debug_country = 'US'
                # and debug_state = 'US-CA' # XXX CA
                # and debug_county = 'US-CA-075' # XXX SF
                # and geohash_bin = '9q8yh' # XXX Glen Canyon / Mt Davidson
            # order by geohash_bin, bin_p_sp desc # XXX Dev
            # order by geohash_bin, species, observation_date_bin # XXX Dev
            # limit 500 # XXX Dev
        )
    select *
    from d
''',
    # max_rows=50,  # XXX
    max_rows=None,  # WARNING
)
    .pipe(df_inspect, lambda df: (df.shape,))
    .pipe(df_inspect, lambda df: humanize.naturalsize_binary(df.memory_usage(deep=True).sum()))
    .pipe(df_inspect, lambda df: df[:50]
        .pipe(lambda df: df if 'bin_p_sp' not in df.columns else df
            .pipe(df_col_map, bin_p_sp=lambda x: '%.3f' % x)
        )
    )
)

Running query...

[36s] cost[$0.1896, 41.7 GB] rows[7907912] size[271.5 MiB] url[https://console.cloud.google.com/bigquery?project=bubo-1&j=bq:US:job_IHY1p5ea63c9plrFXWrWUr8Y0hw6&page=queryresults]

Using .extract i/o .to_dataframe, because size[271.5 MiB] ≥ extract_infer_threshold_mb[100]

Extracting results to uri[gs://potoo-bqq-extract/v0/2019-08-24T23-05-12-990174.csv.gz]...

Peeking .to_dataframe(max_rows=1000) to infer schema for .extract() (disable with extract_infer_schema=False)...

Fetching results from uri[gs://potoo-bqq-extract/v0/2019-08-24T23-05-12-990174.csv.gz]...

[57s]

(7907912, 5)

1.1 GiB

,geohash_bin,observation_date_bin,bin_n_checklists,bin_n_checklists_with_sp,species
0,9my0,0,505,10,YHBL
1,9my0,0,505,10,YHBL
2,9q3h,0,2173,14,WWDO
3,9q78,0,1609,409,RCKI
4,9q90,0,25479,245,ROGO
5,9q90,0,25479,245,ROGO
6,9qd8,0,2115,289,RWBL
7,9qdh,0,1599,47,CLGR
8,9rnh,0,7,3,RLHA
9,9t8s,0,1934,3,CAVI


In [ ]:
# XXX
df_base = (bqq(defs=defs, sql='''
    select distinct
        geohash_at_bits(st_geogpoint(longitude, latitude), 23) as geohash_bin -- 6mi
        # geohash_at_bits(st_geogpoint(longitude, latitude), 21) as geohash_bin -- 12mi
        # geohash_at_bits(st_geogpoint(longitude, latitude), 19) as geohash_bin -- 24mi
        # geohash_at_bits(st_geogpoint(longitude, latitude), 17) as geohash_bin -- 48mi
    # from ebird.obs_sample_001 e # XXX Dev
    from ebird.obs_all_expensive e # WARNING
    where true
        # and category in ('species', 'issf', 'form', 'intergrade')
        # and country_code = 'US'
        # and state_code = 'US-CA' # XXX
        # and county_code = 'US-CA-075' # XXX SF
''',
    max_rows=None,  # WARNING
)
    .pipe(df_inspect, lambda df: (df.shape,))
    .pipe(df_inspect, lambda df: df[:10])
)

Running query...

[9s] cost[$0.0370, 8.1 GB] rows[324744] size[2.2 MiB] url[https://console.cloud.google.com/bigquery?project=bubo-1&j=bq:US:job_zzfA-4GvqWY-lK4SA4N14OfPbGPL&page=queryresults]

Fetching results...

[8s]

(324744, 1)

,geohash_bin
0,66jds
1,u28b0
2,thrrh
3,wdqrd
4,d4kjn
5,9g35w
6,9g4zn
7,thqfd
8,sxk9s
9,6ex30


In [ ]:
# Write .sqlite

# TODO TODO -> mobile payload (cf. payloads.py)
db_path = Path('/tmp') / ('priors-%s.sqlite3' % re.sub(r'[^0-9T]', '-', datetime.utcnow().isoformat()))

with sqla_oneshot_eng_conn_tx(f'sqlite:///{ensure_parent_dir(db_path)}') as conn:
    table = 'priors'
    def size_sql_relation(name: str) -> int:
        df = pd.read_sql(con=conn, sql='''
            select sum(pgsize) as size from dbstat where name=:name
        ''', params=dict(
            name=name,
        ))
        return one(df_rows(df))['size']  # (Careful: df['size'] i/o df.size, else you'll get the row count)
    with log_time_context(f'Mobile: Create table {table}', lambda: humanize.naturalsize_binary(size_sql_relation(table) or 0)):
        (priors_df
            .pipe(df_inspect, lambda df: (df.shape,))
            .pipe(df_inspect, lambda df: humanize.naturalsize_binary(df.memory_usage(deep=True).sum()))
            .pipe(df_inspect, lambda df: df[:10])
            .to_sql(table, conn,
                if_exists='replace',
                index=False,  # Silently drop df index (we shouldn't have any)
                chunksize=1000,  # Safe default for big writes (pd default writes all rows at once -- mem unsafe)
            )
        )

!ls -sh {db_path}

# TODO Smoothing for geohash stuff

In [ ]:
# XXX
df_rel = (
    pd.DataFrame(columns=['x', 'y'], data=(
        (x, y)
        for x in df_base.geohash_bin
        for y in geoh.str_expand(x, 21)  # str_expand i/o str_neighbors so we include (x, x) in relation
    ))
    .pipe(df_inspect, lambda df: (df.shape,))
    .pipe(df_inspect, lambda df: df[:10])
)

(1316175, 2)

,x,y
0,sxk9h,sxk80
1,sxk9h,sxk8h
2,sxk9h,sxk90
3,sxk9h,sxk9h
4,sxk9h,sxkb0
5,sxk9h,sxkc0
6,sxk9h,sxkd0
7,sxk9h,sxkdh
8,sxk9h,sxkf0
9,w60u0,w60eh


In [ ]:
# TODO TODO Create tables to map geohash to its expands, for smoothing
#   - Vary precision to do the kind of smoothing we want: prec=n -> prec=bits+2 -> expands (x9) -> prec=bits (x4)
#   - Q: Do we need all geohashes, or should we restrict just to US ones? (from join with ebird table)
# precision_bits = 1   # len[   2] time[   0s +    7s] bq[   24b,     4 rows]
# precision_bits = 3   # len[   8] time[   0s +    7s] bq[  288b,    48 rows]
# precision_bits = 5   # len[  32] time[   0s +    7s] bq[ 1.4kb,   240 rows]
# precision_bits = 13  # len[  8k] time[   0s +    8s] bq[ 713kb,   73k rows]
# precision_bits = 15  # len[ 32k] time[   2s +   11s] bq[ 2.8mb,  293k rows]
# precision_bits = 17  # len[128k] time[  10s +   24s] bq[13.5mb,  1.8m rows]
# precision_bits = 19  # len[512k] time[  50s +   68s] bq[  54mb,  4.7m rows]
# precision_bits = 21  # len[  2m] time[ 262s +  287s] bq[ 252mb, 18.8m rows]
# precision_bits = 23  # len[  8m] time[1656s + 1013s] bq[ 1.0gb, 75.4m rows]
xs = geoh.all_geohashes_with_expand(precision_bits)
df = (
    pd.DataFrame(
        columns=['x', 'y'],
        data=list(tqdm(total=len(xs), iterable=(
            (k, v)
            for k, vs in xs.items()
            for v in vs
        ))),
    )
    .pipe(df_inspect, lambda df: (df.shape,))
    .pipe(df_inspect, lambda df: df[:10])
    .pipe(df_to_bq,
        table='geoh.all_geohashes_with_expand_precision_bits_%(precision_bits)s' % locals(),
        overwrite=True,
    )
)

100%|██████████████████████████████████████████████████████████████████████| 2097152/2097152 [04:22<00:00, 7995.36it/s]

18862080it [00:06, 2904837.99it/s]

(18862080, 2)

,x,y
0,00000,00000
1,00000,0000h
2,00000,00010
3,00000,0001h
4,00000,pbpbh
5,00000,pbpch
6,0000h,00000
7,0000h,0000h
8,0000h,00010
9,0000h,0001h


Inferred schema[{'x': 'string', 'y': 'string'}] from dtypes[{'x': dtype('O'), 'y': dtype('O')}]

Creating table[geoh.all_geohashes_with_expand_precision_bits_21] with schema[{'x': 'string', 'y': 'string'}]...

Uploading df to gs[gs://potoo-bqq/load/v0/2019-08-25T04-44-58-356172.csv]...

Loading table[geoh.all_geohashes_with_expand_precision_bits_21] from gs[gs://potoo-bqq/load/v0/2019-08-25T04-44-58-356172.csv]...

[287s] rows[18862080] size[251.8 MiB]

In [ ]:
# XXX
bqq(show_query=True, sql='''
    select
        count(distinct x),
        count(*)
    from geoh.all_geohashes_with_expand_precision_bits_%(precision_bits)s
''' % locals())